# Santiago De Chile Delays

Let's import all the tools we need for our data explaration

In [9]:
# !pip install --upgrade pip --user
!pip install -r ../requirements.txt --user

  Using cached pandas-1.3.5-cp39-cp39-win_amd64.whl (10.2 MB)
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)


In [10]:
import pandas as pd

Let's first go over the provided data and see what valuable insights we can find.